In [27]:
#Here: function to handle optimal splits:

import numpy as np
import scipy.io
from Tree import BinaryTree, Node

#get matlab data as nested dictionary
image = scipy.io.loadmat('MysteryImage.mat')

#extract compressed matrix from dictionaries

#columns: y-axis; rows: x-axis; vals: [r,g,b]
Y = image['cols']
X = image['rows']
V = image['vals']

# number of pixels
n = np.shape(X)[0]

print(X)

In [28]:
#split s means we check all y-values to see if they're above or below the line
'''
split cost functions
Desc: finds cost of a split
parameters: 
s: split value
c1: mean color left/below split [r,g,b]
c2: mean color right/above split [r,g,b]
'''
def hor_split_cost(s, c1, c2):
    cost = 0
    for i in range (0,n):
        if(Y[i] <= s):
            cost += np.linalg.norm(V[i]-c1)**2
        else:
            cost += np.linalg.norm(V[i]-c1)**2
    return cost

def vert_split_cost(s,c1,c2):
    cost = 0
    for i in range (0,n):
        if(X[i] <= s):
            cost += np.linalg.norm(V[i]-c1)**2
        else:
            cost += np.linalg.norm(V[i]-c1)**2
    return cost


In [29]:
'''
Regression function
Desc: does tree regression on the image data
parameters: 
MAX_ITTER: maximum number of itterations
X_interval: [a,b] lower and upper boundaries of interval
Y_interval: [a,b] lower and upper boundaries of interval  

Return Values: 
False: Failure

'''

MAX_ITTER = 2
itter = 0
def regression(X_interval, Y_interval):
    #assemble list of pts
    points_in_range = []
    for i in range(0,n):
        if(X[i] in range(X_interval[0], X_interval[1]) and Y[i] in range(Y_interval[0], Y_interval[1])):
            points_in_range.append(i)
    #if(len(points_in_range) < 0):
    #    return False
    s = points_in_range[0]
    min_cost = float('inf')
    # Direction: True = horizontal; False = vertical
    direction = True
    for i in points_in_range:
        #left mean
        c1 = np.zeros((1,3))
        #right mean
        c2 = np.zeros((1,3))
        #Horzontal split:
        count_1 = 0
        for j in points_in_range:
            if(X[j] <= X[i]):
                c1 += V[j]
                count_1 += 1
            else:
                c2 += V[j]
        #average
        if(count_1 > 0):
            c1 = 1/count_1 * c1
        if(len(points_in_range) - count_1 > 0):
            c2 = 1/(len(points_in_range) - count_1) * c2
        #left mean
        c1 = np.zeros((1,3))
        #right mean
        c2 = np.zeros((1,3))

        hor_cost = hor_split_cost(i,c1,c2)
        # if option lower cost, set to new optimal split value & save info about split
        if(hor_cost < min_cost):
            s = i
            min_cost = hor_cost
            direction = True

        #Repeat for vertical split:
        count_1 = 0
        for j in points_in_range:
            if(Y[j] <= Y[i]):
                c1 += V[j]
                count_1 += 1
            else:
                c2 += V[j]
        #average
        if(count_1 > 0):
            c1 = 1/count_1 * c1
        if(len(points_in_range) - count_1 > 0):
            c2 = 1/(len(points_in_range) - count_1) * c2

        hor_cost = hor_split_cost(i,c1,c2)
        if(hor_cost < min_cost):
            s = i
            min_cost = hor_cost
            direction = False

        #End For loop

    #setting up tree structure:
    node = Node(s, direction, c1, c2)
    itter +=1

    #check if terminal node:
    if(itter < MAX_ITTER):        
        if(direction):
            #Horizontal split:
            #We need to do the thing
            node.left = regression(X_interval, [Y_interval[0], Y[s]])
            node.right = regression(X_interval, [Y[s], Y_interval[1]])
        else:
            #Vertical Split:
            node.left = regression([Y_interval[0], Y[s]], Y_interval)
            node.right = regression([Y[s],Y_interval[1]], Y_interval)
    print(f'itteration: {itter}')
    #return node
    return node

In [30]:
root = regression([0, 1456],[0, 2592])
tree = BinaryTree(root)

UnboundLocalError: local variable 'itter' referenced before assignment